In [1]:
import os
from langchain_core.tools import StructuredTool

from dotenv import load_dotenv
from langchain_openai import ChatOpenAI


load_dotenv()

api_key = os.getenv("OPENAI_API_KEY")

model = ChatOpenAI(model="gpt-3.5-turbo")

In [2]:
from web3 import Web3
from eth_account import Account

# Connect to Ethereum blockchain
WORLDCHAIN_URL = os.getenv("WORLDCHAIN_RPC_URL")
web3 = Web3(Web3.HTTPProvider(WORLDCHAIN_URL))

# Check connection
if not web3.is_connected():
    raise ConnectionError("Failed to connect to the blockchain.")

# Wallet setup
PRIVATE_KEY = os.getenv("WALLET_PRIVATE_KEY")

assert PRIVATE_KEY is not None, "You must set PRIVATE_KEY environment variable"
assert PRIVATE_KEY.startswith("0x"), "Private key must start with 0x hex prefix"

ACCOUNT =  Account.from_key(PRIVATE_KEY)


In [7]:
from langchain.agents import Tool
from langchain.prompts import PromptTemplate
from langchain.agents import create_react_agent, AgentExecutor,AgentType, initialize_agent

# Define tools for the agent
def get_wallet_balance(*args):
    balance = web3.eth.get_balance(ACCOUNT.address)
    return f"Wallet Balance: {web3.from_wei(balance, 'ether')} ETH"

#print(ACCOUNT.address)
#balanche= (web3.eth.get_balance(ACCOUNT.address))
#print(web3.from_wei(balanche, 'ether'))


# Tools list
tools = [
    Tool(name="Check Balance", func=get_wallet_balance, description="Get the wallet's current balance."),
]
# Define an agent prompt
prompt = PromptTemplate.from_template(
        """You are a crypto wallet assistant. if you can not answer the Question you are asked use the tools provided to you.""")

# Initialize the agent
agent = initialize_agent(
  tools,
  llm=model,
  agent=AgentType.CHAT_ZERO_SHOT_REACT_DESCRIPTION,
  verbose=False,
)


In [ ]:
agent.run("what is my wallet balance")

'Wallet Balance: 0.999999818964994442 ETH'